# Playground for Mutation Testing with Execution Taints

This jupyter notebook demonstrates the usage of our prototype library that implements execution taints for python.

As multiprocessing is limited in jupyter notebooks, all examples will use the non-forking version.

We start with importing the functions used in the AST rewritten examples and some functions to make our library work in
a notebook context.

In [1]:
from shadow import reinit, t_final_exception_test, t_wrap, t_combine, t_wait_for_forks, t_get_killed, t_cond, t_assert, \
                   t_logical_path, t_seen_mutants, t_masked_mutants, ShadowVariable, t_class, t_active_mutants, t_sv

Define some functions to make it easy to compare which mutations have been killed.

In [2]:
def gen_killed(strong):
    return {
        'strong': set(strong),
    }


def get_killed():
    t_wait_for_forks()
    results = t_get_killed()
    return {
        'strong': set(results['strong']),
    }

The basic component is a tainted variable (called a `ShadowVariable` internally), it combines the different values caused by mutations into one variable.

In [9]:
print("To implement the library transparently global state needs to be kept.")
print("Using the reinit function this global state can be reset.")
print("The 'shadow' execution mode enables the non-forking, non-memoizing execution mode for the library.")
print("Atexit is needed to correctly gather results on the end of programs, this is not needed here.")
reinit(execution_mode='shadow', no_atexit=True)

print()
print("The basic component is a tainted variable, let us create a simple one:")
tainted_int = t_combine({0: 0, 1: 1})
print(tainted_int)

print()
print("Assert that the expected value is zero, this is true for mainline (0) but not for mutation 1.")
t_assert(tainted_int == 0)

print()
print("This results in mutation 1 being marked as strongly killed:")
killed = get_killed()
print(killed)

assert killed == gen_killed({1})

To implement the library transparently global state needs to be kept.
Using the reinit function this global state can be reset.
The 'shadow' execution mode enables the non-forking, non-memoizing execution mode for the library.
Atexit is needed to correctly gather results on the end of programs, this is not needed here.

The basic component is a tainted variable, let us create a simple one:
ShadowVariable({1: 1, 0: 0})

Assert that the expected value is zero, this is true for mainline (0) but not for mutation 1.

This results in mutation 1 being marked as strongly killed:
{'strong': {1}}


Variables can also be simply wrapped adding only the mainline.

In [11]:
reinit(execution_mode='shadow', no_atexit=True)

print()
print("Wrapping a variable:")
tainted_int = t_sv(0)
print(tainted_int)
tainted_list = t_sv([])
print(tainted_list)


Wrapping a variable:
ShadowVariable({0: 0})
ShadowVariable({0: []})


Functions need to be wrapped to correctly handle execution.

`@t_wrap` provides all functionality needed to wrap a function.

To correctly handle control flow we need to wrap conditionals, this functionality is provided by `t_cond`.
If the boolean values differ the execution will be forked in the forking version and re-execution of the function
is done in the non-forking version (which is used in this notebook).

In [31]:
@t_wrap
def add_to_list(data, val):
    ii = 0
    while t_cond(ii < val):
        data.append(ii)
        ii += 1
    print(f"Following mutation {t_logical_path()}, data at end of execution: {data}")

In the next example create a list and update it using the newly defined function.

In [33]:
reinit(execution_mode='shadow', no_atexit=True)

print("Initialize variables")
tainted_int = t_combine({0: 0, 1: 1, 2: 2})
tainted_list = t_sv([])
print(tainted_int)
print(tainted_list)

print()
print("Execute function:")
add_to_list(tainted_list, tainted_int)
print("The results are merged in the wrapper and list is updated:")
print(tainted_list)

Initialize variables
ShadowVariable({1: 1, 2: 2, 0: 0})
ShadowVariable({0: []})

Execute function:
Following mutation 0, data at end of execution: ShadowVariable({0: []})
Following mutation 1, data at end of execution: ShadowVariable({0: [0]})
Following mutation 2, data at end of execution: ShadowVariable({0: [0, 1]})
The results are merged in the wrapper and list is updated:
ShadowVariable({0: [], 1: [0], 2: [0, 1]})


Another more complex example illustrating the execution order for nested functions.

The number after the function name is the currently followed path. The inner function is re-executed until all currently active mutations are evaluated.

In [60]:
@t_wrap
def inner(tainted_int):
    print(f"inner {t_logical_path()}:                     {tainted_int}")
    if t_cond(tainted_int == 1):
        print(f"inner {t_logical_path()} is equal 1:          {tainted_int}")
        tainted_int -= 1
    else:
        print(f"inner {t_logical_path()} is not equal 1:      {tainted_int}")
        tainted_int += 1
    print(f"inner {t_logical_path()} res:                 {tainted_int}")
    return tainted_int

@t_wrap
def func(tainted_int):
    print(f"func  {t_logical_path()}:                     {tainted_int}")
    if t_cond(tainted_int <= 1):
        print(f"func  {t_logical_path()} is less equal 1:     {tainted_int}")
        tainted_int += 1
        tainted_int = inner(tainted_int)
    else:
        print(f"func  {t_logical_path()} is not less equal 1: {tainted_int}")
        tainted_int -= 1
        tainted_int = inner(tainted_int)
    print(f"func  {t_logical_path()} res:                 {tainted_int}")
    return tainted_int

reinit(execution_mode="shadow", no_atexit=True)
var = t_combine({0: 0, 1: 1, 2: 2, 3: 3})
res = func(var)
print(f"Final result: {res}")
t_assert(res == 0)
assert get_killed() == gen_killed([1, 3])

func  0:                     ShadowVariable({1: 1, 2: 2, 3: 3, 0: 0})
func  0 is less equal 1:     ShadowVariable({1: 1, 2: 2, 3: 3, 0: 0})
inner 0:                     ShadowVariable({1: 2, 0: 1})
inner 0 is equal 1:          ShadowVariable({1: 2, 0: 1})
inner 0 res:                 ShadowVariable({0: 0})
inner 1:                     ShadowVariable({1: 2, 0: 1})
inner 1 is not equal 1:      ShadowVariable({1: 2, 0: 1})
inner 1 res:                 ShadowVariable({0: 2, 1: 3})
func  0 res:                 ShadowVariable({0: 0, 1: 3})
func  2:                     ShadowVariable({2: 2, 3: 3, 0: 0})
func  2 is not less equal 1: ShadowVariable({2: 2, 3: 3, 0: 0})
inner 2:                     ShadowVariable({2: 1, 3: 2, 0: -1})
inner 2 is equal 1:          ShadowVariable({2: 1, 3: 2, 0: -1})
inner 2 res:                 ShadowVariable({0: -2, 2: 0})
inner 3:                     ShadowVariable({3: 2, 0: -1})
inner 3 is not equal 1:      ShadowVariable({3: 2, 0: -1})
inner 3 res:             

The prototype also supports user defined classes. This is supported by the `t_class` function wrapper, this transparently wraps attributes and method calls to created objects.

Currently the objects are duplicated for evaluation of further mutants, while computationally expensive this eases the implementation effort.
Performance can be improved for example by using a copy on write mechanism and copying fields individually.

In [73]:
@t_class
class BankAccount:
    balance: int
    overdrawn: bool

    def __init__(self, initial_balance: int):
        self.balance = t_combine({(0): lambda : initial_balance, (1): lambda : initial_balance != 1, (2): lambda : initial_balance + 1, (3): lambda : initial_balance * 2})
        self.overdrawn = t_combine({(0): lambda : False})
        self.update_overdrawn()

    def __repr__(self):
        return f"BankAccount(balance={self.balance}, overdrawn={self.overdrawn})"

    def update_overdrawn(self) ->None:
        if t_cond(self.balance >= 0):
            self.overdrawn = t_combine({(0): lambda : False})
        else:
            self.overdrawn = t_combine({(0): lambda : True, (10): lambda : True != 1})

    def deposit(self, amount: int) ->None:
        self.balance = t_combine({(0): lambda : self.balance + amount, (13): lambda : self.balance - amount, (14): lambda : self.balance * amount, (16): lambda : self.balance % amount, (17): lambda : self.balance << amount, (18): lambda : self.balance >> amount, (19): lambda : self.balance | amount, (20): lambda : self.balance ^ amount, (21): lambda : self.balance & amount, (22): lambda : self.balance // amount})
        self.update_overdrawn()

    def withdraw(self, amount: int) ->None:
        self.balance = t_combine({(0): lambda : self.balance - amount, (23): lambda : self.balance + amount, (24): lambda : self.balance * amount, (26): lambda : self.balance % amount, (27): lambda : self.balance << amount, (28): lambda : self.balance >> amount, (29): lambda : self.balance | amount, (30): lambda : self.balance ^ amount, (31): lambda : self.balance & amount, (32): lambda : self.balance // amount})
        self.update_overdrawn()

    def is_overdrawn(self) ->bool:
        return self.overdrawn


@t_wrap
def bank_example() ->None:
    my_account = BankAccount(10)
    t_assert(my_account.balance == 10)
    t_assert(my_account.overdrawn == False)
    print(my_account)
    print()

    my_account.deposit(5)
    t_assert(my_account.balance == 15)
    t_assert(my_account.overdrawn == False)
    print(my_account)
    print()

    my_account.withdraw(200)
    t_assert(my_account.balance == -185)
    t_assert(my_account.overdrawn == True)
    print(my_account)


reinit(execution_mode="shadow", no_atexit=True)
bank_example()
assert get_killed() == gen_killed([ 1, 2, 3, 10, 13, 14, 16, 17, 18, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32])

ShadowVariable({1: BankAccount(balance=True, overdrawn=False), 2: BankAccount(balance=11, overdrawn=False), 3: BankAccount(balance=20, overdrawn=False), 0: BankAccount(balance=10, overdrawn=False)})

ShadowVariable({13: BankAccount(balance=5, overdrawn=False), 14: BankAccount(balance=50, overdrawn=False), 16: BankAccount(balance=0, overdrawn=False), 17: BankAccount(balance=320, overdrawn=False), 18: BankAccount(balance=0, overdrawn=False), 19: BankAccount(balance=15, overdrawn=False), 20: BankAccount(balance=15, overdrawn=False), 21: BankAccount(balance=0, overdrawn=False), 22: BankAccount(balance=2, overdrawn=False), 0: BankAccount(balance=15, overdrawn=False)})

ShadowVariable({19: BankAccount(balance=-185, overdrawn=True), 20: BankAccount(balance=-185, overdrawn=True), 23: BankAccount(balance=215, overdrawn=False), 24: BankAccount(balance=3000, overdrawn=False), 26: BankAccount(balance=15, overdrawn=False), 27: BankAccount(balance=2410407066388485413312943138511743903783304490674189